In [114]:
from django_for_jupyter import init_django
init_django()

In [115]:
# tạo dataframe content chứa dữ liệu cho gợi ý dựa trên thể loại
from home.models import Book_Topic, Book
from home.models import ContentBook
import pandas as pd
books = Book.objects.all().order_by('book_id')
bookTopic = Book_Topic.objects.prefetch_related('topic_id').order_by('book_id')
check = True
dics = []
for book in books:
    content = book.book_title
    topics = bookTopic.filter(book_id = book.book_id)
    for topic in topics:
        content+=' '+topic.topic_id.topic_name
    dic = {'book_id': book.book_id,
            'content': content
           }
    dics.append(dic) 
book_df = pd.DataFrame(dics)
print(book_df)

    book_id                                            content
0      3001  CDMA capacity and quality optimization code di...
1      3002  Image databases : Search and retrieval of digi...
2      3003  Practical handbook on image processing for sci...
3      3004  Analytics, data science, & artificial intellig...
4      3005  Composing cyberspace : Identity, community, an...
..      ...                                                ...
72     3073  Giáo trình đảm bảo chất lượng phần mềm Softwar...
73     3074  Data engineering Fuzzy mathematics in systems ...
74     3075  Digital image processing Image processing Digi...
75     3076  Digital image processing using MATLAB Image pr...
76     3095  MySQL/PHP database applications Database desig...

[77 rows x 2 columns]


In [129]:
from home.models import ContentBook
import pandas as pd
bookContent = ContentBook.objects.all().order_by('book_id')
dics = []
for book in bookContent:
    dic = {
        'book_id': book.book_id,
        'content': book.content
    }
    dics.append(dic)
book_df = pd.DataFrame(dics)
print(book_df)


    book_id                                            content
0      3001  CDMA capacity and quality optimization code di...
1      3002  Image databases : Search and retrieval of digi...
2      3003  Practical handbook on image processing for sci...
3      3004  Analytics, data science, & artificial intellig...
4      3005  Composing cyberspace : Identity, community, an...
..      ...                                                ...
72     3073  Giáo trình đảm bảo chất lượng phần mềm Softwar...
73     3074  Data engineering Fuzzy mathematics in systems ...
74     3075  Digital image processing Image processing Digi...
75     3076  Digital image processing using MATLAB Image pr...
76     3095  MySQL/PHP database applications Database desig...

[77 rows x 2 columns]


In [117]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text


In [130]:
enlish_stop_words = text.ENGLISH_STOP_WORDS
vietnamese_stop_words = ["và", "là", "của", "những", "với", "từ", "một", "được", "khi", "đã", "cho", "vì", "ở", "này", "giáo", "trình", "lập", "trình"]
combine_stop_words = list(enlish_stop_words) + vietnamese_stop_words


In [119]:

book_tfidf = TfidfVectorizer(stop_words=combine_stop_words)
book_df['content'] = book_df['content'].fillna('')
book_content_matrix = book_tfidf.fit_transform(book_df['content'])
book_content_matrix.shape

cosine_similarity = linear_kernel(book_content_matrix, book_content_matrix)

In [127]:
import pickle

# Lưu TF-IDF Vectorizer
with open('./home/recommend/book_tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(book_tfidf, f)

with open('./home/recommend/book_cosine_similarity.pkl', 'wb') as f:
    pickle.dump(cosine_similarity, f)

In [128]:
from home.models import ContentBook
import pickle
with open('./home/recommend/book_tfidf_vectorizer.pkl', 'rb') as f:
    book_tfidf = pickle.load(f)
bookContents = ContentBook.objects.all().order_by('book_id')
book_df = pd.DataFrame(bookContents)   
book_content_matrix = book_tfidf.fit_transform(book_df['content'])  

KeyError: 'content'

In [77]:
# Thao tac tinh toan va dua ra goi y
choice = 67
similarity_scores = list(enumerate(cosine_similarity[choice-1]))
similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
similarity_scores = similarity_scores[1:6]

# Get the similar books index
books_index = [i[0] for i in similarity_scores]
books = Book.objects.all().order_by('book_id')
# printing the top 5 most similar books using integer-location based indexing (iloc)
print(f'The choice is: {books[choice-1].book_id} {books[choice-1].book_title}')
for i in books_index:
    print(f'{books[i].book_id} - {books[i].book_title} ')


The choice is: 3067 Giáo trình lập trình cho thiết bị di động
3021 - iOS 9 Application Development in 24 hours 
3015 - IOS Programming the big nerd ranch guide : The Big Nerd Ranch guide 
3053 - Giáo trình lập trình hướng đối tượng Java 
3072 - Giáo trình lập trình .Net 
3063 - Giáo trình lập trình hướng đối tượng 


In [68]:
# Giả sử bạn có dữ liệu về sách mới
new_books_df['content'] = new_books_df['content'].fillna('')

# Vector hóa nội dung sách mới
new_book_content_matrix = book_tfidf.transform(new_books_df['content'])

# Cập nhật ma trận cosine similarity
new_cosine_similarity = linear_kernel(new_book_content_matrix, book_content_matrix)

# Gộp ma trận cũ và mới lại để cập nhật toàn bộ hệ thống
cosine_similarity = linear_kernel(book_content_matrix, book_content_matrix)

# Lưu lại các dữ liệu đã cập nhật
with open('book_tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(book_tfidf, f)

with open('book_cosine_similarity.pkl', 'wb') as f:
    pickle.dump(cosine_similarity, f)


In [1]:
from django_for_jupyter import init_django
init_django()

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text



In [133]:
# Khởi tạo lại tfids và cosine
from home.models import ContentBook, Book, Book_Topic
bookContents = ContentBook.objects.all().order_by('book_id').values('book_id', 'content')
book_df = pd.DataFrame(bookContents)

book_tfidf = TfidfVectorizer(stop_words=combine_stop_words)
book_df['content'] = book_df['content'].fillna('')
book_content_matrix = book_tfidf.fit_transform(book_df['content'])

cosine_similarity = linear_kernel(book_content_matrix, book_content_matrix)

import pickle

# Lưu TF-IDF Vectorizer
with open('./home/recommend/book_tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(book_tfidf, f)

with open('./home/recommend/book_cosine_similarity.pkl', 'wb') as f:
    pickle.dump(cosine_similarity, f)


In [143]:
from home.models import ContentBook, Book, Book_Topic
import pandas as pd
latest_book = Book.objects.latest('book_id')
content = latest_book.book_title
topics = Book_Topic.objects.filter(book_id=latest_book.book_id).select_related('topic_id')
for topic in topics:
    content+=' '+ topic.topic_id.topic_name
print(content)
# newContent = ContentBook(book = latest_book, content = content)
# newContent.save()
newContentDic ={
    'book_id': latest_book.book_id,
    'content': content
}
newContent_df = pd.DataFrame(newContentDic, index=[0])
print(newContent_df['content'][0])


Database systems the complete book Database design (Thiết kế database) Database management (Quản trị database)
Database systems the complete book Database design (Thiết kế database) Database management (Quản trị database)


In [ ]:
from home.models import ContentBook
from sklearn.metrics.pairwise import linear_kernel
import pickle
from scipy.sparse import vstack

# open tfidf.pkl to load book_tfidf
with open('./home/recommend/book_tfidf_vectorizer.pkl', 'rb') as f:
    book_tfidf = pickle.load(f)
    
# load tạo dataframe cho ContentBook cũ  
bookContents = ContentBook.objects.all().order_by('book_id').values('book_id', 'content')
book_df = pd.DataFrame(bookContents)
book_df['content'] = book_df['content'].astype(str)

    
newContent_df['content'] = newContent_df['content'].fillna('')
newContent_df['content'] = newContent_df['content'].astype(str)

# Vector hóa nội dung sách mới
book_content_matrix = book_tfidf.fit_transform(book_df['content']) 
newContent_matrix = book_tfidf.transform(newContent_df['content'])

# Gộp ma trận nội dung sách cũ và mới lại
updatedContent_matrix = vstack([book_content_matrix, newContent_matrix])
    
# Cập nhật ma trận cosine similarity cho toàn bộ hệ thống
cosine_similarity = linear_kernel(updatedContent_matrix, updatedContent_matrix)


# Lưu TF-IDF Vectorizer
with open('./home/recommend/book_tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(book_tfidf, f)

with open('./home/recommend/book_cosine_similarity.pkl', 'wb') as f:
    pickle.dump(cosine_similarity, f)

In [14]:
# ham update content book khi update bôok
# lay ra title, topics của cuốn sách mới nhất được cập nhật thông qua created_at
# chạy lại vòng lặp topic và cập nhật content dựa vào book_id
from home.models import ContentBook, Book, Book_Topic
import pandas as pd
lasted_update = Book.objects.latest('created_at')

content = lasted_update.book_title
content = str(content)
topics = Book_Topic.objects.filter(book_id_id = lasted_update.book_id).select_related('topic_id')
for topic in topics:
    print(str(topic.topic_id.topic_name))
    content+=' '+ str(topic.topic_id.topic_name)
    print(content)
content_update = ContentBook.objects.get(book_id = lasted_update.book_id)
content_update.content = content
content_update.save()

updateContentDic = {
    'book_id': lasted_update.book_id,
    'content': content
}
updateContent_df = pd.DataFrame(updateContentDic, index=[0])
print(updateContent_df)


Data structure (cấu trúc dữ liệu)
Perl database programming Data structure (cấu trúc dữ liệu)
Database structure
Perl database programming Data structure (cấu trúc dữ liệu) Database structure
   book_id                                            content
0     3079  Perl database programming Data structure (cấu ...
